# Intro to `pandas` for machine learning

We'll explore the Pandas package for simple data handling tasks using geoscience data examples. 

Pandas introduces the concept of a `DataFrame` in Python. If you're familiar with R, it's pretty much the same idea! Useful cheat sheet [here](https://www.datacamp.com/community/blog/pandas-cheat-sheet-python#gs.59HV6BY)

The main purpose of Pandas is to allow easy manipulation of data in tabular form. Perhaps the most important idea that makes Pandas great for data science, is that it will always preserve **alignment** between data and labels.

In [1]:
import pandas as pd

The most common data structure in Pandas is the `DataFrame`, which is a 2D structure that can hold various types of Python objects indexed by an `index` array (or multiple `index` arrays). Columns are usually labelled as well using strings. A column on its own is a different data type, called a `Series`.

An easy way to think about a `DataFrame` is if you imagine it as an Excel spreadsheet.

Let's define one using a small dataset:

In [2]:
data =  [[2.13, 'sandstone'],
         [3.45, 'limestone'],
         [2.45, 'shale']]
data

[[2.13, 'sandstone'], [3.45, 'limestone'], [2.45, 'shale']]

Make a `DataFrame` from `data`

In [3]:
df = pd.DataFrame(data, columns=['velocity', 'lithology'])
df

,velocity,lithology
0,2.13,sandstone
1,3.45,limestone
2,2.45,shale


Accessing the data is a bit more complex than in the numpy array cases but for good reasons

In [4]:
df['lithology']

0    sandstone
1    limestone
2        shale
Name: lithology, dtype: object

In [ ]:
# df[0]  # Can't get at rows like this.

In [5]:
df.loc[0:1]  # Inclusive slice, unlike anything else in Python or NumPy

,velocity,lithology
0,2.13,sandstone
1,3.45,limestone


In [ ]:
df['id'] = [101, 102, 103]
df = df.set_index('id')
df.head()

In [ ]:
df.loc[0]

In [6]:
# Skip this unless folks ask for it.
df.iloc[0:1]  # Works like NumPy, rarely needed

,velocity,lithology
0,2.13,sandstone


In [7]:
df['velocity'] < 3

0     True
1    False
2     True
Name: velocity, dtype: bool

In [8]:
df.loc[df['velocity'] < 3]  # Loc provides *selectors* like NumPy arrays.

,velocity,lithology
0,2.13,sandstone
2,2.45,shale


In [9]:
df.loc[df['velocity'] < 3, 'lithology']  # Both rows and columns.


0    sandstone
2        shale
Name: lithology, dtype: object

## Adding data

Add more data (row wise). This is adding a new record.

In [10]:
df.loc[99] = [3.5, 'dolomite']

In [11]:
df.loc[3] = [2.6, 'shale']
df

,velocity,lithology
0,2.13,sandstone
1,3.45,limestone
2,2.45,shale
99,3.50,dolomite
3,2.60,shale


To add a new feature or attribute we want to add a new Series.

Add a new column with a "complete" list, array or series.

In [12]:
df['method'] = ['ultrasonic', 'tomographic', 'acoustic', 'petrophysical', 'ultrasonic']
df

,velocity,lithology,method
0,2.13,sandstone,ultrasonic
1,3.45,limestone,tomographic
2,2.45,shale,acoustic
99,3.50,dolomite,petrophysical
3,2.60,shale,ultrasonic


Alternatively, you can broadcast a value or calculation.

In [13]:
df['source'] = 'Agile'
df['velocity_si'] = df['velocity'] * 1000
df

,velocity,lithology,method,source,velocity_si
0,2.13,sandstone,ultrasonic,Agile,2130.0
1,3.45,limestone,tomographic,Agile,3450.0
2,2.45,shale,acoustic,Agile,2450.0
99,3.50,dolomite,petrophysical,Agile,3500.0
3,2.60,shale,ultrasonic,Agile,2600.0



## Exercise

* Add a new boolean Series (with `True` or `False` values) `is_carbonate`. If a record has a `velocity_si` greater than 3000 it should be `True`.
* Add a new record (row) with `id` 87, `velocity_si` of 2225, a `lithology` of sandstone, and a 'ultrasonic' `method`. Use your own name as the `source`. For other columns, feel free to make up values as needed.
* Create a subset of the current dataframe with only the `velocity_si`, `lithology`, `method` and `source` columns and name it `df2`.
* Adjust the `velocity_si` values of measurements only made using the 'ultrasonic' `method` in `df2` by subtracting 175 m/s.

In [14]:
df['is_carbonate'] = df['velocity_si'] > 3000
df.loc[87] = [2.225, 'sandstone', 'ultrasonic', 'Agile', 2225, False]
df2 = df[['lithology', 'source', 'method', 'velocity_si']]

In [15]:
df2.loc[df2.method=='ultrasonic', 'velocity_si'] = df.velocity_si - 175
df2

,lithology,source,method,velocity_si
0,sandstone,Agile,ultrasonic,1955.0
1,limestone,Agile,tomographic,3450.0
2,shale,Agile,acoustic,2450.0
99,dolomite,Agile,petrophysical,3500.0
3,shale,Agile,ultrasonic,2425.0
87,sandstone,Agile,ultrasonic,2050.0


## Column types

A Series has a `dtype` associated with it, which is the type (eg: int, string, float, &c) of the data stored within it. A given Series can only have one dtype, so the more general one is used, for example a Series with both integer and float values will be considered a float Series. These affect what can be done to a given Series (for example a string Series will not have a mean associated with it, while Series with floats or ints will).

In [16]:
values = [53, 58, 60, '62', '70', '74']

In [17]:
s = pd.Series(values)
s

0    53
1    58
2    60
3    62
4    70
5    74
dtype: object

In [18]:
s * 2

0     106
1     116
2     120
3    6262
4    7070
5    7474
dtype: object

In [19]:
s.astype(float)

0    53.0
1    58.0
2    60.0
3    62.0
4    70.0
5    74.0
dtype: float64

In [20]:
s = s.astype(float)

In [21]:
s * 2

0    106.0
1    116.0
2    120.0
3    124.0
4    140.0
5    148.0
dtype: float64

### Strings
pandas has two ways to store strings. From the [docs](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes):
> * `object` dtype, which can hold any Python object, including strings.
> * `StringDtype`, which is dedicated to strings.
> 
> Generally, we recommend using StringDtype. See [Text data types](https://pandas.pydata.org/docs/user_guide/text.html#text-types) for more.
> Finally, arbitrary objects may be stored using the object dtype, but should be avoided to the extent possible (for performance and interoperability with other libraries and methods. See [object conversion](https://pandas.pydata.org/docs/user_guide/basics.html#basics-object-conversion)).
> A convenient dtypes attribute for DataFrame returns a Series with the data type of each column.

A string-like column can have things that we normally do to strings done to it:

In [22]:
df2['lithology'] * 2

0     sandstonesandstone
1     limestonelimestone
2             shaleshale
99      dolomitedolomite
3             shaleshale
87    sandstonesandstone
Name: lithology, dtype: object

In [23]:
df2['lithology'].str.upper()

0     SANDSTONE
1     LIMESTONE
2         SHALE
99     DOLOMITE
3         SHALE
87    SANDSTONE
Name: lithology, dtype: object

Mixed values can cause some problems though:

In [24]:
density = [
    'Medium',
    'Low',
    'Med',
    3214,
    'High',
    'Very high'
]

First we'll try accepting the default `object` data type:

In [25]:
s = pd.Series(density)
s

0       Medium
1          Low
2          Med
3         3214
4         High
5    Very high
dtype: object

In [26]:
s.str.upper() # [str(ii).upper() for ii in s]

0       MEDIUM
1          LOW
2          MED
3          NaN
4         HIGH
5    VERY HIGH
dtype: object

In [27]:
s.str.isdigit()

0    False
1    False
2    False
3      NaN
4    False
5    False
dtype: object

In [28]:
s.str.replace('Med$', 'Medium', regex=True)

0       Medium
1          Low
2       Medium
3          NaN
4         High
5    Very high
dtype: object

Note that the indices of this list do not match the DataFrame index, so we have to explicitly pass the index to the Series constructor. (Try leaving `index=df.index` out.)

If we use a string type instead, then some things will work more smoothly.

In [29]:
s = pd.Series(density, dtype='string')  # Or pd.StringDtype()

In [30]:
s.dtype

string[python]

Using this does not, however, give us direct access to string methods — e.g. you still need `str` here — but at least things work more consistently:

In [31]:
s.str.replace('Med$', 'Medium', regex=True)

0       Medium
1          Low
2       Medium
3         3214
4         High
5    Very high
dtype: string

In [32]:
s.str.isdigit()

0    False
1    False
2    False
3     True
4    False
5    False
dtype: boolean

In [33]:
s * 2

0          MediumMedium
1                LowLow
2                MedMed
3              32143214
4              HighHigh
5    Very highVery high
dtype: string

### Categories

It makes sense to use the [Pandas `categorical` type](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) for categorical variables. These are good for a few cases:

> * A string variable consisting of only a few different values (for example lithologies in a well log). Converting such a string variable to a categorical variable will save some memory.
> * The lexical order of a variable is not the same as the logical order (“one”, “two”, “three”). By converting to a categorical and specifying an order on the categories, sorting and min/max will use the logical order instead of the lexical order.
> * As a signal to other Python libraries that this column should be treated as a categorical variable (e.g. to use suitable statistical methods or plot types).

In [34]:
cols = ['lithology', 'method']
df[cols] = df[cols].apply(pd.Categorical)

df.dtypes

velocity         float64
lithology       category
method          category
source            object
velocity_si      float64
is_carbonate        bool
dtype: object

We can now use those to do some useful things, for example:

In [35]:
df['method']

0        ultrasonic
1       tomographic
2          acoustic
99    petrophysical
3        ultrasonic
87       ultrasonic
Name: method, dtype: category
Categories (4, object): ['acoustic', 'petrophysical', 'tomographic', 'ultrasonic']

In [36]:
df[['method', 'lithology']].describe()

,method,lithology
count,6,6
unique,4,4
top,ultrasonic,sandstone
freq,3,2


## Reading and writing files

Pandas also reads files from disk in tabular form ([here](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html)'s a list of all the formats that it can read and write). A very common one is Excel's `.xlsx` format, so let's load one!

The data is the same as used in this study: http://www.kgs.ku.edu/PRS/publication/2003/ofr2003-30/index.html

From that poster:

> The Panoma Field (2.9 TCF gas) produces from Permian Council Grove Group marine carbonates and nonmarine silicilastics in the Hugoton embayment of the Anadarko Basin. It and the Hugoton Field, which has produced from the Chase Group since 1928, the top of which is 300 feet shallower have combined to produce 27 TCF gas, making it the largest gas producing area in North America. Both fields are stratigraphic traps with their updip west and northwest limits nearly coincident. Maximum recoveries in the Panoma are attained west of center of the field. Deeper production includes oil and gas from Pennsylvanian Lansing-Kansas City, Marmaton, and Morrow and the Mississippian.

For Excel files, we can load specific sheets by passing the `sheet_name` argument:

In [37]:
df = pd.read_excel('https://geocomp.s3.amazonaws.com/data/Panoma_Field_Permian-RAW.xlsx', sheet_name='data')
df.head()

,Well Name,Depth,Formation,RelPos,Marine,GR,ILD,DeltaPHI,PHIND,PE,Facies,Completion Date,Index,Source
0,SHRIMPLIN,851.3064,A1 SH,1.000,1,77.45,4.613176,9.9,11.915,4.6,3.0,2010-03-26,63,KGS
1,SHRIMPLIN,851.4588,A1 SH,0.979,1,78.26,4.581419,14.2,12.565,4.1,3.0,2010-03-26,64,KGS
2,SHRIMPLIN,851.6112,A1 SH,0.957,1,79.05,4.549881,14.8,13.050,3.6,3.0,2010-03-26,65,KGS
3,SHRIMPLIN,851.7636,A1 SH,0.936,1,86.10,4.518559,13.9,13.115,3.5,3.0,2010-03-26,66,KGS
4,SHRIMPLIN,851.9160,A1 SH,0.915,1,74.58,4.436086,13.5,13.300,3.4,3.0,2010-03-26,67,KGS


Without it, we get the first sheet, which in this case is not the data that we want, but it may still be useful:

In [38]:
pd.read_excel('https://geocomp.s3.amazonaws.com/data/Panoma_Field_Permian-RAW.xlsx')

,Column Name,Description,Units
0,Index,Index for sorting records,NaN
1,Well Name,Name of the well that the record is from.,NaN
2,Depth,Measured depth below KB,NaN
3,Formation,Which formation the record is from. See accomp...,NaN
4,RelPos,Position of the record relative to a facies in...,NaN
5,Marine,Whether a record is of a marine rock (1) or no...,NaN
6,GR,Gamma Ray - natural radioactivity.,Unit: API units
7,DeltaPHI,Difference between the Neutron Porosity and th...,Unit: %
8,ILD,Induction Log Deep - Deep formation resistivity,Unit: Ohm.m
9,PHIND,Nuclear Density for porosity. Average of DPHI ...,Unit: API units


We can also load all the worksheets as a dictionary by passing `sheet_name=None`, which gives us a dictionary of DataFrames, with the key being the sheet name:

In [39]:
pd.read_excel('https://geocomp.s3.amazonaws.com/data/Panoma_Field_Permian-RAW.xlsx', sheet_name=None)

{'metadata':         Column Name                                        Description  \
 0             Index                          Index for sorting records   
 1         Well Name          Name of the well that the record is from.   
 2             Depth                            Measured depth below KB   
 3         Formation  Which formation the record is from. See accomp...   
 4            RelPos  Position of the record relative to a facies in...   
 5            Marine  Whether a record is of a marine rock (1) or no...   
 6                GR                 Gamma Ray - natural radioactivity.   
 7          DeltaPHI  Difference between the Neutron Porosity and th...   
 8               ILD    Induction Log Deep - Deep formation resistivity   
 9             PHIND  Nuclear Density for porosity. Average of DPHI ...   
 10               PE                             Photoelectric factor.    
 11           Facies  Numerical code for the Facies. Derived from co...   
 12  Completi

Other formats are usually loaded in a similarly way, using the `pd.read_*` pattern: `pd.read_csv`, `pd.read_csv` and so on.


## Exercise

* Select only the `Well Name`, `Depth` and `GR` columns.
* Select only the records for the `LUKE G U` well. How many rows are there?
* Select the `Depth` and `GR` columns only for the `LUKE G U` well.
* Select the `Depth` and `GR` column for the `LUKE G U` well with depth greater than 850 m. What is the mean value of `GR`?

In [40]:
df[['Well Name', 'Depth', 'GR']]

,Well Name,Depth,GR
0,SHRIMPLIN,851.3064,77.450
1,SHRIMPLIN,851.4588,78.260
2,SHRIMPLIN,851.6112,79.050
3,SHRIMPLIN,851.7636,86.100
4,SHRIMPLIN,851.9160,74.580
...,...,...,...
4894,CRAWFORD,3158.5000,86.078
4895,CRAWFORD,3159.0000,88.855
4896,CRAWFORD,3159.5000,90.490
4897,CRAWFORD,3160.0000,90.975


In [41]:
df.loc[df['Well Name'] == 'LUKE G U']

,Well Name,Depth,Formation,RelPos,Marine,GR,ILD,DeltaPHI,PHIND,PE,Facies,Completion Date,Index,Source
1386,LUKE G U,795.6804,A1 SH,1.000,1,74.90,6.053409,9.3,11.75,4.084,3.0,2012-10-01,1449,KGS
1387,LUKE G U,795.8328,A1 SH,0.981,1,83.80,5.559043,12.0,13.10,3.501,3.0,2012-10-01,1450,KGS
1388,LUKE G U,795.9852,A1 SH,0.962,1,86.97,5.321083,12.9,12.55,3.400,3.0,2012-10-01,1451,KGS
1389,LUKE G U,796.1376,A1 SH,0.943,1,84.43,5.105050,13.2,12.00,3.400,3.0,2012-10-01,1452,KGS
1390,LUKE G U,796.2900,A1 SH,0.925,1,78.51,5.105050,11.8,11.40,3.400,3.0,2012-10-01,1453,KGS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842,LUKE G U,865.6320,C LM,0.074,2,79.48,4.886524,7.7,26.15,2.500,8.0,2012-10-01,1905,KGS
1843,LUKE G U,865.7844,C LM,0.059,2,62.88,4.886524,12.6,23.70,2.700,8.0,2012-10-01,1906,KGS
1844,LUKE G U,865.9368,C LM,0.044,2,41.04,4.295364,13.7,10.65,3.200,8.0,2012-10-01,1907,KGS
1845,LUKE G U,866.0892,C LM,0.029,2,33.99,3.689776,3.2,3.90,3.800,8.0,2012-10-01,1908,KGS


In [42]:
df.loc[df['Well Name'] == 'LUKE G U', ['Depth', 'GR']]
# feel free to add .plot(x='GR', y='Depth', figsize=(2,6))

,Depth,GR
1386,795.6804,74.90
1387,795.8328,83.80
1388,795.9852,86.97
1389,796.1376,84.43
1390,796.2900,78.51
...,...,...
1842,865.6320,79.48
1843,865.7844,62.88
1844,865.9368,41.04
1845,866.0892,33.99


In [43]:
df.loc[(df['Well Name'] == 'LUKE G U') & (df['Depth'] > 850), ['Depth', 'GR']]

,Depth,GR
1743,850.0872,72.69
1744,850.2396,73.82
1745,850.3920,77.10
1746,850.5444,78.49
1747,850.6968,78.59
...,...,...
1842,865.6320,79.48
1843,865.7844,62.88
1844,865.9368,41.04
1845,866.0892,33.99


In [44]:
df.loc[(df['Well Name'] == 'LUKE G U') & (df['Depth'] > 850), ['GR']].mean()

GR    67.665385
dtype: float64

In [ ]:
# This might be a bit much, but it can also show the utility of getting a subset out.
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(2,6))
df.loc[(df['Well Name'] == 'LUKE G U') & (df['Depth'] > 850),
       ['GR', 'Depth']
      ].plot(x='GR', y='Depth', ax=ax)
ax.axvline(67.67, c='r')
ax.invert_yaxis()

## Writing Data Out

Writing data out is similarly simple to reading it in, using one of the range of `to_*` functions. In this case, we will go with a simple `csv` format:

In [45]:
df.to_csv('../data/Panoma_Field_Permian_RAW.csv', index=False)

If we are comfortable with SQL, or have an existing database, we may wish to write our dataframe as a table there. We will use the Python implementation of [sqlite](https://www.sqlite.com/index.html), [sqlite3](https://docs.python.org/3/library/sqlite3.html). If you have an existing database you may prefer to look at [SQLalchemy](https://docs.sqlalchemy.org/) to create the connection instead:

In [46]:
import sqlite3

In [47]:
connection = sqlite3.connect('../data/panoma.db')
df.to_sql('panoma_raw', con=connection, if_exists='replace', index=False)

4899

You could use `read_sql` to get data from a SQL database instead of reading a file.

<hr />

<p style="color:gray">©2022 Agile Geoscience. Licensed CC-BY.</p>